# Building NLP Products Tutorial

> "You shall know a word by the company it keeps." ~ John R. Firth

![img](https://cdn.shopify.com/s/files/1/0867/3580/products/vinyl_decal_hello_words_cloud_ig4779_1800x1800.jpg?v=1571439560)

## Learning Outcomes

By the end of this tutorial you will
1. Have a better understanding of natural language processing and some of its applications.
2. Be able to create recommendation systems based on text similarity.
3. Be able to conduct topic modeling on your own corpus.
4. Understand how to put together a simple app using panel.

## Table of Contents

1. Overview
2. The Data
3. Flash NLP Intro
4. Cleaning
5. Recommendation System
6. Topic Modeling
7. Summary

## 1. Overview

With have been given a random corpus of articles taken from Wikipedia and our task is to come up with two products, a recommendations systems and a set of topic that best explains the model. This will help you and anyone else who picks up this notebook, understand the Wikipedia corpus better.

In [1]:
import json, nltk, re, spacy
import pandas as pd, numpy as np
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity
import panel as pn
from concurrent.futures import ThreadPoolExecutor

pn.extension()

%load_ext autoreload
%autoreload 2

It is possible that you will need the following packages in order to move forward. Please copy the two lines below, paste them in a new cell and run it.

```python
nltk.download('wordnet')
nltk.download('punkt')
```

## 2. The Data

The data consist of Wikipedia articles plus some additional columns inside a JSON file. Here is the schema.

| Column | Content |
|--------|---------|
|title |Title of article|
|url | Url of article|
|abstract | Abstract of article|
|body_text | Text inside article|
|body_html | Article inside HTML|

Before we do any data cleaning, let's read in the data and explore it a bit.

In [2]:
%%time

data_list = [] # empty list that will hold a line of data for us

for line in open('data.jsonl', 'r'):
    data_list.append(json.loads(line)) # read in line by line

CPU times: user 9.61 s, sys: 3.19 s, total: 12.8 s
Wall time: 14.1 s


Let's see how many articles we have and then examine the very first one.

In [3]:
len(data_list), data_list[0]

(64844,
 {'title': 'Wikibooks: Romanian/Lesson 9',
  'url': 'https://en.wikibooks.org/wiki/Romanian/Lesson_9',
  'abstract': '==Băuturi/Beverages==',
  'body_text': 'Băuturi/Beverages[edit\xa0| edit source]\nTea\xa0: Ceai\nMilk\xa0: Lapte\nWater\xa0: Apă (If you are in Romania, and want to ask for plain tap water, ask for apă plată.)\nSparkling water\xa0: Apă minerală\nSoda\xa0: Sifon\nBeer\xa0: Bere\nWine\xa0: Vin\nMâncăruri/Foods[edit\xa0| edit source]\nBread\xa0: Pâine\nPotato\xa0: Cartof\nMashed potatoes\xa0: Piure de cartofi\nFrench fries\xa0: Cartofi prăjiți\nCheese (To put on bread)\xa0: Caşcaval\nFeta cheese\xa0: Brânza\nSteak\xa0: Friptură\nSoup\xa0: Supă\nChicken\xa0: Pui\nBeef\xa0: Vacă\nDuck\xa0: Rață\nPork\xa0: Porc\nOranges\xa0: Portocale\nTomatoes\xa0: Roșii\nToast\xa0: Pâine prăjită (lit. "Fried bread".)\nApple\xa0: Măr\nTacâmuri/Eating utensils[edit\xa0| edit source]\nKnife\xa0: Cuţit\nFork\xa0: Furculiţă\nSpoon\xa0: Lingură\nTeaspoon\xa0: Linguriţă\nGlass\xa0: Pahar\n

Now that we have a nice list of dictionaries, we can create a pandas DataFrame. You can think of pandas DataFrames as as Excel spreadsheets we can use to hold and manipulate our data for us.

In [4]:
df = pd.DataFrame(data_list)
df.head()

,title,url,abstract,body_text,body_html
0,Wikibooks: Romanian/Lesson 9,https://en.wikibooks.org/wiki/Romanian/Lesson_9,==Băuturi/Beverages==,Băuturi/Beverages[edit | edit source]\nTea : C...,"<div class=""mw-parser-output""><h2><span id=""B...."
1,Wikibooks: Karrigell,https://en.wikibooks.org/wiki/Karrigell,Karrigell is an open Source Python web framewo...,Karrigell is an open Source Python web framewo...,"<div class=""mw-parser-output""><p>Karrigell is ..."
2,Wikibooks: The Pyrogenesis Engine/0 A.D./GuiSe...,https://en.wikibooks.org/wiki/The_Pyrogenesis_...,====setupUnitPanel====,setupUnitPanel[edit | edit source]\nHelper fun...,"<div class=""mw-parser-output""><h4><span class=..."
3,Wikibooks: LMIs in Control/pages/Exterior Coni...,https://en.wikibooks.org/wiki/LMIs_in_Control/...,== The Concept ==,Contents\n\n1 The Concept\n2 The System\n3 The...,"<div class=""mw-parser-output""><div id=""toc"" cl..."
4,Wikibooks: Laptop Computer Models/Dell/Latitud...,https://en.wikibooks.org/wiki/Laptop_Computer_...,= Dell Latitude D830 =,Contents\n\n1 Dell Latitude D830\n\n1.1 CPU\n1...,"<div class=""mw-parser-output""><div id=""toc"" cl..."


## 3. Flash NLP Intro

We can use the `.loc[index, column]` method on our dataframe, select one column and one row using a comma to separate both, and examine a prettier version of the text using the python function `pprint()`.

In [5]:
random_article = df.loc[10, 'body_text']
pprint(random_article)

('This Wikibooks page is a fact sheet and analysis on the article "Habitual '
 'physical activity in children and adolescents with cystic fibrosis" about '
 'how exercise is related to the disease Cystic Fibrosis.\n'
 '\n'
 'Contents\n'
 '\n'
 '1 Background of this research\n'
 '2 Where is the research from\xa0?\n'
 '3 What kind of research was this?\n'
 '4 What did the research involve?\n'
 '\n'
 '4.1 Pulmonary Function testing\n'
 '4.2 Pros / Cons of this test\n'
 '\n'
 '\n'
 '5 What were the basic results?\n'
 '6 What conclusion can we take from this research\xa0?\n'
 '7 Practical Advice\n'
 '8 Further information/ Resources\n'
 '\n'
 '8.1 Cystic Fibrosis Australia\n'
 "8.2 Cystic Fibrosis's National Ambassador Nathan Charles\n"
 '\n'
 '\n'
 '9 References\n'
 '\n'
 '\n'
 '\n'
 'Background of this research[edit\xa0| edit source]\n'
 'The research was about the effects of taking part in exercise constantly or '
 'making it a habit in the population of children and teens that are sever

Notice how the review above is quite messy and it has a lot of characters that, for all intents and purposes, will not be useful for our analysis. Let's examine a cleaner version of the article above by running it through spaCy's tokenizer. When we tokenize a document, we are separating all of its content into each of its components, i.e. words, numbers, punctiations and the like, to make it easier to process and to run computations on it.

For this part, we will load an english model, instantiate it and pass an example article through it. You may need to run the cell below first to download the english model.

In [6]:
# python -m spacy download en_core_web_sm

In [7]:
nlp = spacy.load('en_core_web_sm')

In [8]:
parsed_article = nlp(random_article)

In [9]:
parsed_article

This Wikibooks page is a fact sheet and analysis on the article "Habitual physical activity in children and adolescents with cystic fibrosis" about how exercise is related to the disease Cystic Fibrosis.

Contents

1 Background of this research
2 Where is the research from ?
3 What kind of research was this?
4 What did the research involve?

4.1 Pulmonary Function testing
4.2 Pros / Cons of this test


5 What were the basic results?
6 What conclusion can we take from this research ?
7 Practical Advice
8 Further information/ Resources

8.1 Cystic Fibrosis Australia
8.2 Cystic Fibrosis's National Ambassador Nathan Charles


9 References



Background of this research[edit | edit source]
The research was about the effects of taking part in exercise constantly or making it a habit in the population of children and teens that are severing from the genetic condition cystic Fibrosis.
What is  Cystic Fibrosis
It is a genetic condition, affecting lungs and digestion. Unfortunately, there is no 

Notice how much nicer our article looks like now.

We can also grab sentences and view them one by one we wanted to using the attribute `.sents` and the built in python function `next()`. Conversely, we can add it to a loop and show each of the sentences in an article.

In [10]:
next(enumerate(parsed_article.sents))

(0,
 This Wikibooks page is a fact sheet and analysis on the article "Habitual physical activity in children and adolescents with cystic fibrosis" about how exercise is related to the disease Cystic Fibrosis.)

In [11]:
for num, sentence in enumerate(parsed_article.sents):
    print(f"Sentence #{num}:\n {sentence}\n")

Sentence #0:
 This Wikibooks page is a fact sheet and analysis on the article "Habitual physical activity in children and adolescents with cystic fibrosis" about how exercise is related to the disease Cystic Fibrosis.

Sentence #1:
 

Contents

1 Background of this research
2 Where is the research from ?

Sentence #2:
 
3

Sentence #3:
 What kind of research was this?

Sentence #4:
 
4

Sentence #5:
 What did the research involve?

Sentence #6:
 

4.1 Pulmonary Function testing
4.2 Pros / Cons of this test


5

Sentence #7:
 What were the basic results?

Sentence #8:
 
6

Sentence #9:
 What conclusion can we take from this research ?

Sentence #10:
 
7 Practical Advice
8 Further information/ Resources

8.1 Cystic Fibrosis Australia
8.2 Cystic Fibrosis's National Ambassador Nathan Charles


9 References



Background of this research[edit

Sentence #11:
  | edit source]


Sentence #12:
 The research was about the effects of taking part in exercise constantly or making it a habit in the 

We can also have a look at the different kinds of entities in an article. These entities can be a person (called PERSON), and number (called CARDINAL), a geopolitical entity (called GPE), etc.

In [12]:
for num, entity in enumerate(parsed_article.ents):
    print(f"Entity #{num}: {entity} -- {entity.label_}\n")

Entity #0: 2 -- CARDINAL

Entity #1: 3 -- CARDINAL

Entity #2: 4.2 -- CARDINAL

Entity #3: Pros / Cons -- ORG

Entity #4: 5 -- CARDINAL

Entity #5: 6 -- CARDINAL

Entity #6: 8 -- CARDINAL

Entity #7: Australia -- GPE

Entity #8: 8.2 -- CARDINAL

Entity #9: Nathan Charles -- PERSON

Entity #10: 9 -- CARDINAL

Entity #11: Fibrosis -- PRODUCT

Entity #12: Cystic Fibrosis -- ORG

Entity #13: 1 -- CARDINAL

Entity #14: 3300 -- CARDINAL

Entity #15: American -- NORP

Entity #16: Pittsburgh -- GPE

Entity #17: Two -- CARDINAL

Entity #18: David Michael Orenstein -- PERSON

Entity #19: CF -- GPE

Entity #20: Austria -- GPE

Entity #21: Two -- CARDINAL

Entity #22: David Michael -- PERSON

Entity #23: Patricia -- PERSON

Entity #24: the Journal of Paediatric Pulmonology -- ORG

Entity #25: three -- CARDINAL

Entity #26: 60 -- CARDINAL

Entity #27: 7–17 years -- DATE

Entity #28: 30 -- CARDINAL

Entity #29: 18 -- CARDINAL

Entity #30: 12 -- CARDINAL

Entity #31: 30 -- CARDINAL

Entity #32: 17 --

We can also check weather a word is a stopword or a punctuation, or we can even lemmatize our articles. Lemmatization is a way of taking the root of a word and bringing similar words to a common denominator, for example, was will become be and most plural words will be singular words.

In [13]:
# here we are taking out of the parsed article each token
token_text = [token.text for token in parsed_article]

# here we are lemmatizing each word possible
token_lemmas = [token.lemma_ for token in parsed_article]

# stopwords are very common so here we will extract a variable that will tell us whether
# a word is a stopword or not
token_stop = [token.is_stop for token in parsed_article]

token_punc = [token.is_punct for token in parsed_article]

# we will now add all three to a dataframe and display it without assigning it to a variable
pd.DataFrame(zip(token_text, token_lemmas, token_punc, token_stop), columns=['Original Text', 'Lemmatized Text', 'Punctuations', 'stopwords']).head(50)

,Original Text,Lemmatized Text,Punctuations,stopwords
0,This,this,False,True
1,Wikibooks,Wikibooks,False,False
2,page,page,False,False
3,is,be,False,True
4,a,a,False,True
5,fact,fact,False,False
6,sheet,sheet,False,False
7,and,and,False,True
8,analysis,analysis,False,False
9,on,on,False,True


## 4. Cleaning

Let's start by checking if our dataset contains any missin values, and then evaluate the amount of memory we are currently using from our machine.

In [14]:
df.isna().sum()

title        0
url          0
abstract     0
body_text    0
body_html    0
dtype: int64

In [15]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64844 entries, 0 to 64843
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      64844 non-null  object
 1   url        64844 non-null  object
 2   abstract   64844 non-null  object
 3   body_text  64844 non-null  object
 4   body_html  64844 non-null  object
dtypes: object(5)
memory usage: 4.4 GB


Over 4 GBs is a lot and it is almost certain that most of that comes from the `body_html` column. Let's get rid of it since we already have the `body_text` column, and then let's evaluate again how much data we are using.

In [16]:
df.drop('body_html', axis=1, inplace=True)

In [17]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64844 entries, 0 to 64843
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      64844 non-null  object
 1   url        64844 non-null  object
 2   abstract   64844 non-null  object
 3   body_text  64844 non-null  object
dtypes: object(4)
memory usage: 1007.2 MB


Excellent, let's deal with the titles now. It seems that every abstract starts with `Wikibooks:` so let's check if this is the case and if so, let's take that out.

In [18]:
df.title.str.startswith('Wikibooks: ').sum()

64844

In [19]:
df['clean_title'] = df.title.str.replace('Wikibooks: ', '')

Perfect! Let's now extract the `body_text` and `abstract` columns and normalize them. This means we will the `nltk` library to,
- tokenize the documents,
- take out anything that is not a word or a number,
- convert to lower case,
- strip the spaces around the words,
- remove stopwords (we will use spaCy's list of stopwords for this),
- and then join the cleaned tokens back together.

In [20]:
articles = df['body_text'].values
abstracts = df['abstract'].values

In [21]:
from spacy.lang.en.stop_words import STOP_WORDS
len(STOP_WORDS), STOP_WORDS

(326,
 {"'d",
  "'ll",
  "'m",
  "'re",
  "'s",
  "'ve",
  'a',
  'about',
  'above',
  'across',
  'after',
  'afterwards',
  'again',
  'against',
  'all',
  'almost',
  'alone',
  'along',
  'already',
  'also',
  'although',
  'always',
  'am',
  'among',
  'amongst',
  'amount',
  'an',
  'and',
  'another',
  'any',
  'anyhow',
  'anyone',
  'anything',
  'anyway',
  'anywhere',
  'are',
  'around',
  'as',
  'at',
  'back',
  'be',
  'became',
  'because',
  'become',
  'becomes',
  'becoming',
  'been',
  'before',
  'beforehand',
  'behind',
  'being',
  'below',
  'beside',
  'besides',
  'between',
  'beyond',
  'both',
  'bottom',
  'but',
  'by',
  'ca',
  'call',
  'can',
  'cannot',
  'could',
  'did',
  'do',
  'does',
  'doing',
  'done',
  'down',
  'due',
  'during',
  'each',
  'eight',
  'either',
  'eleven',
  'else',
  'elsewhere',
  'empty',
  'enough',
  'even',
  'ever',
  'every',
  'everyone',
  'everything',
  'everywhere',
  'except',
  'few',
  'fifteen',

In [22]:
def normalize_doc(doc):
    """
    This function normalizes your list of documents by taking only
    words, numbers, and spaces in between them. It then filters out
    stop words.
    """
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    tokens = nltk.word_tokenize(doc)
    filtered_tokens = [token for token in tokens if token not in STOP_WORDS]
    doc = ' '.join(filtered_tokens)
    return doc

In [23]:
normalize_doc(random_article)

'wikibooks page fact sheet analysis article habitual physical activity children adolescents cystic fibrosis exercise related disease cystic fibrosis contents 1 background research 2 research 3 kind research 4 research involve 41 pulmonary function testing 42 pros cons test 5 basic results 6 conclusion research 7 practical advice 8 information resources 81 cystic fibrosis australia 82 cystic fibrosiss national ambassador nathan charles 9 references background researchedit edit source research effects taking exercise constantly making habit population children teens severing genetic condition cystic fibrosis cystic fibrosis genetic condition affecting lungs digestion unfortunately cure condition cystic fibrosis cf inherited white population 1 3300 live births diagnosed condition1 research edit edit source research based american childrens hospital pittsburgh cf centre volunteers research included siblings friends hospital employees children condition authors research work department paed

We will also create the same version of the function but without taking the stopwords out or converting to lowecase, to normalize the abstract.

In [24]:
def normalize_abs(doc):
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.strip()
    tokens = nltk.word_tokenize(doc)
    doc = ' '.join([token for token in tokens])
    return doc

In [25]:
normalize_abs(df.loc[10, 'abstract'])

'This Wikibooks page is a fact sheet and analysis on the article Habitual physical activity in children and adolescents with cystic fibrosis about how exercise is related to the disease Cystic Fibrosis'

Since we have about 60k articles, this operation can take quite some time unless we the cleaning process concurrently. We will do this using the `ThreadPoolExecutor()` from the `concurrent.futures` module.

In [26]:
%%time

with ThreadPoolExecutor(max_workers=8) as e:
    processed_articles = list(e.map(normalize_doc, articles))
    processed_abstract = list(e.map(normalize_abs, abstracts))

CPU times: user 14min 15s, sys: 15.9 s, total: 14min 30s
Wall time: 14min 41s


We will add the cleaned versions of the documents back into the dataframe and loop over these while taking the lenght (in characters terms) of each article.

In [27]:
%%time

df['clean_text'] = processed_articles
df['clean_abstract'] = processed_abstract
df['len_clean_text'] = df['clean_text'].apply(len)
df['len_dirty_text'] = df['body_text'].apply(len)

CPU times: user 92.1 ms, sys: 217 ms, total: 309 ms
Wall time: 454 ms


Let's now save our cleaned dataset in case we need to restart our notebook and begin the analysis again. We will also release a bit of memory by getting rid of all the data and variables we have loaded up since the beginning of the notebook.

In [28]:
%%time

df[['url', 'clean_abstract', 'clean_title', 'clean_text', 'len_clean_text', 'len_dirty_text']].to_parquet('clean_data/clean.parquet', compression='snappy')

CPU times: user 2.13 s, sys: 2.5 s, total: 4.64 s
Wall time: 6.55 s


In [29]:
del data_list
del df
del articles
del abstracts
del processed_articles
del processed_abstract

In [30]:
df = pd.read_parquet('clean_data/clean.parquet')

In [31]:
df.head()

,url,clean_abstract,clean_title,clean_text,len_clean_text,len_dirty_text
0,https://en.wikibooks.org/wiki/Romanian/Lesson_9,ButuriBeverages,Romanian/Lesson 9,buturibeveragesedit edit source tea ceai milk ...,632,827
1,https://en.wikibooks.org/wiki/Karrigell,Karrigell is an open Source Python web framewo...,Karrigell,karrigell open source python web framework wri...,953,1250
2,https://en.wikibooks.org/wiki/The_Pyrogenesis_...,setupUnitPanel,The Pyrogenesis Engine/0 A.D./GuiSession,setupunitpaneledit edit source helper function...,146,185
3,https://en.wikibooks.org/wiki/LMIs_in_Control/...,The Concept,LMIs in Control/pages/Exterior Conic Sector Lemma,contents 1 concept 2 system 3 data 4 lmi exter...,3034,11040
4,https://en.wikibooks.org/wiki/Laptop_Computer_...,Dell Latitude D830,Laptop Computer Models/Dell/Latitude D830,contents 1 dell latitude d830 11 cpu 12 memory...,543,617


It wouldn't make any sense to feed to our algorithms articles with zero words, so let's examine the distribution of characters among both, the raw and the clean version of our articles.

In [32]:
df[['len_clean_text', 'len_dirty_text']].describe().T

,count,mean,std,min,25%,50%,75%,max
len_clean_text,64844.0,5727.485195,24572.765181,0.0,436.0,1485.0,4741.25,1260060.0
len_dirty_text,64844.0,8534.831303,36413.955414,0.0,641.0,2235.0,7150.00,1851361.0


In [33]:
df[['len_clean_text', 'len_dirty_text']].skew()

len_clean_text    22.119524
len_dirty_text    21.950325
dtype: float64

Now that we know we have a skewed distribution of characters, let's fix that by setting up a rule. We'll evaluate an article using a tweets' maximum character count, 280 at the time of writing, and filter out all articles with less than that. Let's check how many we have first.

In [34]:
shorter_than_a_tweet = df['len_clean_text'] < 280
shorter_than_a_tweet.sum()

12031

In [35]:
df = df[~shorter_than_a_tweet].copy()

In [36]:
df.shape

(52813, 6)

# 5. Recommendation System

Recommendation systems can come in many different forms and sizes. We can create a system that takes into account the behaviour of other users, or a system that only looks at similar articles or items to make a recommendation. Both are powerful systems and could cover an entire book in their own right, which is why we will focus on the latter category, the one that makes recommendations based on similar articles.

To create our recommendation system we first need to convert our articles into a numerical representation. We do this with a so-called bag of words (bow). BOWs are matrices with the documents in the rows, the terms contained in all documents along the columns, and the frequency with which each term appears in each document along the values. To create this kind of representation we can use `sklearn`'s `CountVectorizer` or `TfidfVectorizer` classes. The latter being the normalized version of the former, i.e. the frequency of a word divided by the amount of documents in which it appears.

To use this classes we first instantiate them, fit the data to them so that they can learn the vocabulary of our corpus, and then we tranform the corpus into a sparse matrix. These sparse matrices hold the location of all non-zero values to make it easier to store the data and compute on it.

In [40]:
%%time

# if you would rather work with a sample of the dataset to see how it works, use the following one
small_df = df.sample(15_000).copy()

# otherwise, use this one
# small_df = df

small_df.head()

CPU times: user 7.29 ms, sys: 1.69 ms, total: 8.98 ms
Wall time: 17.2 ms


,url,clean_abstract,clean_title,clean_text,len_clean_text,len_dirty_text
7498,https://en.wikibooks.org/wiki/Chess_Opening_Th...,Nimzowitsch Defence,Chess Opening Theory/1. e4/1...Nc6,nimzowitsch defence b c d e f g h 8 8 7 7 6 6 ...,2370,3368
23262,https://en.wikibooks.org/wiki/Foundations_and_...,What is Inclusion,Foundations and Assessment of Education/Editio...,inclusion jennifer leonard learning targets st...,7369,10038
12849,https://en.wikibooks.org/wiki/Arabic/Basics_of...,The basics of Conjugation,Arabic/Basics of Conjugation,basics conjugation verb word indicates action ...,2046,3950
32809,https://en.wikibooks.org/wiki/OpenClinica_User...,There are at least two ways to edit a subjects...,OpenClinica User Manual/EditingSubjectDateOfBirth,ways edit subjects date birth fields enrollmen...,696,1207
26394,https://en.wikibooks.org/wiki/Muggles%27_Guide...,Synopsis,Muggles' Guide to Harry Potter/Books/Half-Bloo...,chapter 14 harry potter halfblood prince felix...,6044,9655


In [41]:
%%time

# we first instantiate our class
tf = TfidfVectorizer(min_df=0.035, max_df=0.80)

# we can fit and transform the data in the same step
tfidf_matrix = tf.fit_transform(small_df['clean_text'].values)

# evaluate the shape of our matrix
tfidf_matrix.shape

CPU times: user 8.35 s, sys: 432 ms, total: 8.78 s
Wall time: 8.95 s


(15000, 1527)

We can access our vocabulary with `.get_feature_names()` method.

In [42]:
tf.get_feature_names()

['000',
 '01',
 '05',
 '10',
 '100',
 '1000',
 '101',
 '11',
 '110',
 '111',
 '112',
 '12',
 '121',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '1994',
 '1995',
 '1997',
 '1998',
 '1999',
 '20',
 '200',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '300',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '3d',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '70',
 '71',
 '72',
 '73',
 '75',
 '80',
 '81',
 '82',
 '83',
 '85',
 '90',
 '91',
 '95',
 'ability',
 'able',
 'accept',
 'accepted',
 'access',
 'according',
 'account',
 'accurate',
 'achieve',
 'achieved',
 'act',
 'action',
 'actions',
 'active',
 'activities',
 'activity',
 'acts',
 'actual',
 'actually',
 'add',

The next step is to get the distance between documents and words to see how close and how far, based on words only, are two documents from one another. The `cosine_similarity` similarity function we imported earlier can do this for us, and afterwards, we can create a dataframe to evaluate our results.

**Note:** this operation can take a few minutes if you are using the entire dataset. Grab some ☕️ 😎

In [43]:
%%time

doc_sim = cosine_similarity(tfidf_matrix)

CPU times: user 11.3 s, sys: 3.04 s, total: 14.3 s
Wall time: 16.1 s


In [44]:
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,14990,14991,14992,14993,14994,14995,14996,14997,14998,14999
0,1.000000,0.042184,0.026870,0.035424,0.092974,0.004593,0.060819,0.047123,0.047415,0.151410,...,0.033870,0.027602,0.016078,0.034820,0.086501,0.129373,0.078498,0.027330,0.065882,0.083768
1,0.042184,1.000000,0.033793,0.029057,0.053811,0.028707,0.032493,0.135438,0.025879,0.091861,...,0.014253,0.006832,0.008648,0.069728,0.064579,0.072633,0.114374,0.020482,0.049906,0.059406
2,0.026870,0.033793,1.000000,0.096364,0.064850,0.015511,0.024777,0.043138,0.047576,0.008359,...,0.041212,0.000000,0.005166,0.030692,0.027689,0.023344,0.044208,0.042139,0.014864,0.053904
3,0.035424,0.029057,0.096364,1.000000,0.062843,0.027845,0.106120,0.052245,0.094568,0.132216,...,0.022883,0.000000,0.007499,0.040615,0.019203,0.097300,0.045980,0.011071,0.060020,0.095773
4,0.092974,0.053811,0.064850,0.062843,1.000000,0.049037,0.086212,0.076450,0.102509,0.168761,...,0.049053,0.000000,0.011900,0.078273,0.073642,0.129334,0.103779,0.022742,0.106421,0.323471


In [45]:
doc_sim.shape

(15000, 15000)

The reason we see a 5000x5000 matrix is because both halfs alonside the diagonal like are completely the same.

In [46]:
articles_list = small_df['clean_title'].values
abstract_list = small_df['clean_abstract'].values
articles_list.shape, articles_list

((15000,),
 array(['Chess Opening Theory/1. e4/1...Nc6',
        'Foundations and Assessment of Education/Edition 1/Foundations Table of Contents/Chapter 4/4.1.1',
        'Arabic/Basics of Conjugation', ..., 'C Sharp Programming/Basics',
        'SPM/Design efficiency',
        "Muggles' Guide to Harry Potter/Characters/Rufus Scrimgeour"],
       dtype=object))

Let's now
1. pick a title at random
2. get the index of such title
3. select the corresponding row for such title in our new document similarity dataframe
4. sort the index of such values
5. return the top 5 article titles

In [47]:
from random import choice

In [48]:
a_title = choice(articles_list)
a_title

'Persian/Lesson 8/man'

In [49]:
article_idx = np.where(articles_list == a_title)[0][0]
article_idx

10531

In [50]:
article_similarities = doc_sim_df.iloc[article_idx].values
article_similarities

array([0.06070983, 0.0366481 , 0.20488106, ..., 0.03188959, 0.07477137,
       0.14059992])

In [51]:
# note that we don't select the first one as this should always be one
similar_articles_idxs = np.argsort(-article_similarities)[1:10]
similar_articles_idxs

array([ 1846,  1654, 11081,  8087, 13880,  5011,  4205,   948,  6591])

In [52]:
similar_articles = articles_list[similar_articles_idxs]
pprint(similar_articles.tolist())

['Mario Kart DS/Nintendo WFC',
 'Feminism/Literature/Gender Trouble',
 'Pinyin/Body modification',
 'Pandunia',
 'Interlingua/Curso de conversation/Capitulo 7, Scenas 1 e 2 (anglese)',
 'English in Use/Pronouns',
 'Blender 3D: Noob to Pro/Box Modeling',
 'Guide to Social Activity/Losing fear',
 'Relationships/Flirting']


In [53]:
similar_abstracts = abstract_list[similar_articles_idxs]
pprint(similar_abstracts[2])

'Search Pinyin body'


Lastly, we will create create a mini-dashboard containing,
1. a widget with all of our titles,
2. a function with the steps we followed above,
3. a panel object to store a title, the widget, and the function.

In [54]:
titles = small_df.clean_title.unique().tolist()
title_widget = pn.widgets.Select(value=choice(titles), options=titles, name='Articles')

In [55]:
@pn.depends(title_widget.param.value)
def article_recommender(title_widget):
    
    article_idx = np.where(articles_list == title_widget)[0][0]
    article_similarities = doc_sim_df.iloc[article_idx].values
    similar_title_idxs = np.argsort(-article_similarities)[1:6]
    similar_titles = articles_list[similar_title_idxs]
    
    return pn.Column(*similar_titles, width=600)

In [56]:
text = pn.pane.Markdown(f"# Small Recommendation Engine", style={"color": "#000000"}, width=600, height=50,
                        sizing_mode="stretch_width", margin=(10,10,10,5))

In [57]:
pn.Column(text, title_widget, article_recommender, align='center', width=600, height=300)

Column(align='center', height=300, sizing_mode='fixed', width=600)
    [0] Markdown(str, height=50, margin=(10, 10, 10, 5), sizing_mode='stretch_width', style={'color': '#000000'}, width=600)
    [1] Select(name='Articles', options=['Chess Opening Theory/1. ...], value='Using SPSS and P...)
    [2] ParamFunction(function)

## 6. Topic Modeling

What is topic modeling?

> "In machine learning and natural language processing, a topic model is a type of statistical model for discovering the abstract "topics" that occur in a collection of documents. Topic modeling is a frequently used text-mining tool for discovery of hidden semantic structures in a text body. Intuitively, given that a document is about a particular topic, one would expect particular words to appear in the document more or less frequently: "dog" and "bone" will appear more often in documents about dogs, "cat" and "meow" will appear in documents about cats, and "the" and "is" will appear approximately equally in both." ~ [Wikipedia](https://en.wikipedia.org/wiki/Topic_model)

As with the recommendation engine, topic modeling requires a bag of words for the representation of the data and, in contrast, it requires a topic number as the key parameter for the model.

In [58]:
vectorizer = CountVectorizer(strip_accents = 'unicode', min_df=0.035, max_df=0.80)

In [59]:
bow = vectorizer.fit_transform(small_df['clean_text'].values)
bow

<15000x1527 sparse matrix of type '<class 'numpy.int64'>'
	with 1721035 stored elements in Compressed Sparse Row format>

In [60]:
topics = 20

In [61]:
lda_model = LatentDirichletAllocation(n_components=topics, # number of topics
                                      max_iter=100, # these are the amount of times the algorithm will run
                                      learning_method='online', 
                                      random_state=42, # setting a seed for reproducible results
                                      n_jobs=-1) # this parameter makes sure we use all of the cores in our machine

In [62]:
%%time

lda_model.fit(bow)

CPU times: user 2min 12s, sys: 44.9 s, total: 2min 57s
Wall time: 5min 39s


LatentDirichletAllocation(learning_method='online', max_iter=100,
                          n_components=20, n_jobs=-1, random_state=42)

We will create a function to explore the topics and their words to see if we can tease apart the main idea of a topic.

In [63]:
def show_topics(vectorizer, lda_model, n_words=15):
    """
    This function takes our vectorizer, our model, and a
    number of words to display the topics from our model.
    """
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

Play around with the topic number and the words evaluated to see which amounts makes most sense to you./

In [64]:
show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=10)

[array(['number', 'line', 'value', 'table', 'left', 'right', 'type',
        'color', 'group', '000'], dtype='<U16'),
 array(['students', 'learning', 'school', 'education', 'teachers',
        'student', 'teacher', 'french', 'lessons', 'schools'], dtype='<U16'),
 array(['water', 'cells', 'cell', 'species', 'body', 'found', 'native',
        'form', 'called', 'food'], dtype='<U16'),
 array(['social', 'communication', 'information', 'people', 'project',
        'work', 'group', 'research', 'development', 'process'],
       dtype='<U16'),
 array(['de', 'person', 'rule', 'en', 'day', 'shall', 'said', 'die',
        'years', 'good'], dtype='<U16'),
 array(['edit', 'source', '10', '12', '11', '01', '05', '13', '14', '15'],
       dtype='<U16'),
 array(['edit', 'file', 'source', 'use', 'command', 'select', 'files',
        'set', 'click', 'window'], dtype='<U16'),
 array(['patients', 'vs', 'health', 'research', 'exercise', 'edit',
        'source', 'study', 'treatment', 'disease'], dtype='<U1

In [65]:
terms = sorted(tf.vocabulary_.keys())

In [66]:
bow_docs = pd.DataFrame(tfidf_matrix.toarray(), columns=terms)
bow_docs.head()

,000,01,05,10,100,1000,101,11,110,111,...,writing,written,wrong,www,year,years,yes,york,young,zero
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.044517,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.062116,0.000000,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.011419,0.0,0.0,...,0.0,0.0,0.000000,0.105695,0.015933,0.014772,0.0,0.020498,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.025950,0.0,0.0,...,0.0,0.0,0.046658,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.043771,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.041665,0.000000,0.0,0.000000,0.0,0.0


The components of our model can be found `lda_model.components_` and can help us create different sets of dataframes, namely, terms-to-topics and document-to-topics. The former has as its values the number of times a word is assigned in a topic, and the latter is the probabily of the words in a document being contained in a topic.

In [67]:
topic_term = pd.DataFrame(lda_model.components_.T, index=terms, columns=['topic_' + str(i) for i in range(topics)])
topic_term.tail()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
years,0.050000,261.621643,470.516078,0.050000,1699.465295,34.665490,0.050000,2177.388355,0.050000,206.409780,1615.667630,0.050000,102.254113,2.452544,0.05000,2636.998296,0.050000,125.030167,88.499697,74.956650
yes,0.050000,0.050000,0.050000,0.050000,156.655105,0.050000,249.654607,7.664245,425.534839,0.050000,0.050000,0.050000,393.338383,0.050000,0.05000,0.050000,0.684041,0.050000,0.050000,801.311408
york,0.050000,67.825382,216.732487,0.050380,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,1565.809635,32.125437,0.050000,689.256340,0.05000,488.857784,0.050000,0.050000,0.050000,0.050000
young,0.050000,106.280880,203.950920,0.050268,237.818339,17.909651,0.050000,181.178674,0.050000,0.050000,50.959536,0.050000,0.050000,119.643770,0.05000,1380.485146,0.050000,0.050000,0.050000,0.050000
zero,306.646906,12.470397,0.050000,0.050000,0.050000,0.050000,0.050010,0.050000,36.551402,478.507752,0.050000,475.696931,39.250386,0.050000,1103.53233,0.050000,643.618087,0.050000,30.931622,0.050000


In [68]:
doc_topic = pd.DataFrame(lda_model.transform(tfidf_matrix), index=small_df.clean_title, columns=['topic_' + str(i) for i in range(topics)])
doc_topic.tail(3)

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
clean_title,,,,,,,,,,,,,,,,,,,,
C Sharp Programming/Basics,0.008047,0.008047,0.008047,0.008047,0.008047,0.008047,0.008047,0.008047,0.008047,0.008047,0.008047,0.517662,0.008047,0.008047,0.008047,0.008047,0.337485,0.008047,0.008047,0.008047
SPM/Design efficiency,0.013758,0.013758,0.013758,0.013758,0.013758,0.164907,0.013758,0.126585,0.013758,0.013758,0.013758,0.474623,0.013758,0.013758,0.013758,0.013758,0.013758,0.013758,0.013758,0.013758
Muggles' Guide to Harry Potter/Characters/Rufus Scrimgeour,0.004415,0.004415,0.004415,0.004415,0.004415,0.004415,0.004415,0.004415,0.004415,0.004415,0.004415,0.004415,0.004415,0.004415,0.004415,0.793077,0.004415,0.004415,0.004415,0.127459


Lastly, a good way to examine the output of an LDA model is by visulizing it with nice graphs and for this we have, `pyLDAvis`. Which is a python library for visualizing topic modeling. We first load it with it's sklearn backend while enabling the notebook setting. Next we use `pyLDAvis.sklearn.prepare` and pass in our model, the bag of words, and the fitted vectorizer to get a nice interactive visualization tool.

In [69]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [72]:
pyLDAvis.sklearn.prepare(lda_model, bow, vectorizer)

/Users/ramonpzg/anaconda/envs/vector/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
16    -0.133324 -0.177287       1        1  8.042159
5      0.407312 -0.081612       2        1  7.919757
3     -0.130606  0.132919       3        1  7.665909
15    -0.114827  0.111008       4        1  6.747149
8     -0.115345 -0.019470       5        1  6.649663
12    -0.095043  0.025703       6        1  5.881798
9     -0.124831 -0.060971       7        1  5.840154
17     0.418165 -0.062634       8        1  5.383033
6     -0.055261 -0.101009       9        1  5.354856
10    -0.028367  0.133060      10        1  5.319366
2     -0.128180  0.048354      11        1  4.899498
11    -0.114767 -0.063219      12        1  4.872918
14    -0.065081 -0.247954      13        1  4.228101
7      0.067896  0.111117      14        1  4.120098
18    -0.067582  0.048422      15        1  3.576519
4     -0.021170  0.080516      16        1  3.434930
13     0.267041  0.139942      17        1  3.261592
0     -0.002062 -0.205646      18        1  3.036546
1     -0.001888  0.201778      19        1  2.533337
19     0.037920 -0.013015      20        1  1.232619, topic_info=              Term           Freq          Total Category  logprob  loglift
476           edit  228619.000000  228619.000000  Default  30.0000  30.0000
449   displaystyle   67347.000000   67347.000000  Default  29.0000  29.0000
1297        source  147985.000000  147985.000000  Default  28.0000  28.0000
3               10   60822.000000   60822.000000  Default  27.0000  27.0000
133            age   31829.000000   31829.000000  Default  26.0000  26.0000
...            ...            ...            ...      ...      ...      ...
1429          true     587.363093    7355.384513  Topic20  -4.7694   1.8685
1230       section     566.488122    9018.007936  Topic20  -4.8056   1.6285
772            key     544.077848    6897.236750  Topic20  -4.8460   1.8562
1297        source     849.688267  147985.673652  Topic20  -4.4002  -0.7640
145       analysis     501.366160    4980.448456  Topic20  -4.9277   2.1001

[941 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0         7  0.022670   000
0        10  0.293162   000
0        11  0.028080   000
0        13  0.000773   000
0        14  0.001030   000
...     ...       ...   ...
1526     12  0.150853  zero
1526     13  0.356128  zero
1526     15  0.009548  zero
1526     18  0.097386  zero
1526     19  0.004137  zero

[6572 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 6, 4, 16, 9, 13, 10, 18, 7, 11, 3, 12, 15, 8, 19, 5, 14, 1, 2, 20])

## 7. Summary

Blind Spots

With additional time we could have,
1. Further tweak the parameters of the vectorizers and models;
2. Create visualizations of both, the best topics and the document similarity to find more interesting patters;
3. Take the title of an article out of the body of the article to create a better, less bias representation of the words within a document;
4. Using Pytorch's nn.CosineSimilarity would help a lot with increasing the efficiency of our recommendation system;
5. There should have been a lemmatization step in the preprocessing stage.

Takeaways,
1. Recommendation systems and topic modeling are both unsupervised methods;
2. Recommendation systems can be created with or without users behavioural data;
3. Topic modeling compresses the data into the most important and meaninful words set by you;
4. Creating bags of words requires careful attention to the parameters;
5. Where possible, showcase a model or system in a mini-dashboard or data visualization.